# Seq2Seq - Encoder/Decoder networks
In this exercise we'll have a deeper look into the ability to use multiple RNN's to infer and generate sequences of data.
Specifically we will implement a Encoder-Decoder RNN based for a simple sequence to sequence translation task.
This type of models have shown impressive performance in Neural Machine Translation and Image Caption generation. 

In the encoder-decoder structure one RNN (blue) encodes the input into a hidden representation, and a second RNN (red) uses this representation to predict the target values.
An essential step is deciding how the encoder and decoder should communicate.
In the simplest approach you use the last hidden state of the encoder to initialize the decoder.
This is what we will do in this notebook, as shown here:

![](./images/enc-dec.png)

In this exercise we will translate from the words of number (e.g. 'nine') to the actual number (e.g. '9').
The input for the Encoder RNN consists of words defining the number, whilst the output of such an encoding serves as input for the Decoder RNN that aims to generate generate a number. 
Our dataset is generated and consists of numbers and an End-of-Sentence (EOS) character ('#'). The data we want to generate should be like follows:

```
Examples: 
prediction  |  input
991136#00 	 nine nine one one three six
81771#000 	 eight one seven seven one
3519614#0 	 three five one nine six one four
26656#000 	 two six six five six
60344#000 	 six zero three four four
162885#00 	 one six two eight eight five
78612625# 	 seven eight six one two six two five
9464710#0 	 nine four six four seven one zero
191306#00 	 one nine one three zero six
10160378# 	 one zero one zero six three seven eight
```

Let us define the space of characters and numbers to be learned with the networks:

```
Number of valid characters: 27
'0'=0,	'1'=1,	'2'=2,	'3'=3,	'4'=4,	'5'=5,	'6'=6,	'7'=7,	'8'=8,	'9'=9,	'#'=10,	' '=11,	'e'=12,	'g'=13,	'f'=14,	'i'=15,	'h'=16,	'o'=17,	'n'=18,	's'=19,	'r'=20,	'u'=21,	't'=22,	'w'=23,	'v'=24,	'x'=25,	'z'=26,	
Stop/start character = #
```

All represented characters and numbers as characters, gets mapped to an integer from 0-26. Our total space of valid characters consists of 27.

In [145]:
from data_generator import generate
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from data_generator import generate

device =  "cpu"#torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device in use:", device)

NUM_INPUTS = 27 #No. of possible characters
NUM_OUTPUTS = 11  # (0-9 + '#')

### Hyperparameters and general configs
MAX_SEQ_LEN = 8
MIN_SEQ_LEN = 5
BATCH_SIZE = 8
TRAINING_SIZE = 8000
LEARNING_RATE = 0.003
N_LAYERS = 2
DROPOUT = 0.5

# Hidden size of enc and dec need to be equal if last hidden of encoder becomes init hidden of decoder
# Otherwise we would need e.g. a linear layer to map to a space with the correct dimension
NUM_UNITS_ENC = NUM_UNITS_DEC = 256
HIDDEN_DIM = 512
TEST_SIZE = 100
EPOCHS = 10
TEACHER_FORCING = True
NUM_OF_BATCHES=8

assert TRAINING_SIZE % BATCH_SIZE == 0

Device in use: cpu


For this exercise we won´t worry about data generation, but utilise a built function for this purpose. The function generates random data constained by the 27 characters described above.

The encoder takes as input the embedded text strings generated from the *generate* function as given here above ie. 'nine' would become [18 15 18 12].
Sequeneces are generated at random given settings of minima and maxima length, constrained by the dimensions of the two RNN´s architecture.
We may visualise a subset of the data generated by running the command below

In [146]:
!python data_generator.py

batch  0  starting
batch  0  ended with  1000
batch  1  starting
batch  1  ended with  1000
batch  2  starting
batch  2  ended with  1000
batch  3  starting
batch  3  ended with  1000
batch  4  starting
batch  4  ended with  1000
batch  5  starting
batch  5  ended with  1000
batch  6  starting
batch  6  ended with  1000
batch  7  starting
batch  7  ended with  1000
Generated batch length 8 from 1000 iterations
Number of valid characters: 27
'0'=0,	'1'=1,	'2'=2,	'3'=3,	'4'=4,	'5'=5,	'6'=6,	'7'=7,	'8'=8,	'9'=9,	'#'=10,	'z'=11,	'g'=12,	'i'=13,	'f'=14,	's'=15,	' '=16,	'e'=17,	'o'=18,	'r'=19,	't'=20,	'w'=21,	'n'=22,	'x'=23,	'v'=24,	'u'=25,	'h'=26,	
Stop/start character = #

SAMPLE 0
TEXT INPUTS:			 ['two two two', 'four two nine six', 'six four five three', 'three nine three nine', 'nine zero three', 'two zero zero eight', 'three two five one', 'seven five five', 'eight nine eight five', 'two five four', 'five zero five five', 'nine six four', 'four one six', 'four zero', 'six four', 'zero 

## Let's define the two RNN's



In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size,n_layers, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
            
        self.embedding = nn.Embedding(input_size, self.hidden_size)
        #rnn = nn.GRU
        self.rnn = nn.LSTM(self.hidden_size, self.hidden_size,n_layers, dropout = dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs, hidden):
        # Input shape [batch, seq_in_len]
        # inputs = [inputs[0],inputs[2]]
        inputs = inputs.long()


        # Embedded shape [batch, seq_in_len, embed]
        embedded = self.dropout(self.embedding(inputs))
        #embedded = embedded.view(embedded.shape[0]*embedded.shape[1],embedded.shape[2],embedded.shape[3])
        
        # Output shape [batch, seq_in_len, embed]
        # Hidden shape [1, batch, embed], last hidden state of the GRU cell
        # We will feed this last hidden state into the decoder
        #print(embedded.shape)
       # Reshape our output to match the input shape of our forward pass
        #hidden = hidden.reshape(hidden.shape[0],1, hidden.shape[1],hidden.shape[2])
        #hidden=hidden.unsqueeze_(0)
        #print(hidden[1].shape)
        #view(len(sentence), 1, -1)
        #print(test.shape)
        outputs,hidden = self.rnn(embedded)
        #print(hidden.shape)
        return outputs,hidden

    def init_hidden(self, batch_size):
        init = torch.zeros(1, batch_size, self.hidden_size, device=device)
        return init


In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,n_layers, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        #rnn = nn.GRU
        self.rnn = nn.LSTM(self.hidden_size, self.hidden_size,n_layers, dropout = dropout, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs, hidden, output_len, teacher_forcing=False):
        # Input shape: [batch, output_len]
        # Hidden shape: [seq_len=1, batch_size, hidden_dim] (the last hidden state of the encoder)

        if teacher_forcing:
            dec_input = inputs
            embed = self.dropout(self.embedding(dec_input))   # shape [batch, output_len, hidden_dim]
            out, hidden = self.rnn(embed,hidden)
            #out, hidden = self.rnn(embed, hidden)
            out = self.out(out)  # linear layer, out has now shape [batch, output_len, output_size]
            output = F.log_softmax(out, -1)
        else:
            # Take the EOS character only, for the whole batch, and unsqueeze so shape is [batch, 1]
            # This is the first input, then we will use as input the GRU output at the previous time step
            dec_input = inputs[:, 0].unsqueeze(1)

            output = []
            for i in range(output_len):
                embed = self.dropout(self.embedding(dec_input))
                out, hidden= self.rnn(embed,hidden)
                #out, hidden = self.rnn(self.embedding(dec_input), hidden)
                out = self.out(out)  # linear layer, out has now shape [batch, 1, output_size]
                out = F.log_softmax(out, -1)
                output.append(out.squeeze(1))
                out_symbol = torch.argmax(out, dim=2)   # shape [batch, 1]
                dec_input = out_symbol   # feed the decoded symbol back into the recurrent unit at next step

            output = torch.stack(output).permute(1, 0, 2)  # [batch_size x seq_len x output_size]

        return output

The learned representation from the *Encoder* gets propagated to the *Decoder* as the final hidden layer in the *Encoder* network is set as initialisation for the *Decoder*'s first hidden layer.

In [0]:
def forward_pass(encoder, decoder, x, t, t_in, criterion, max_t_len, teacher_forcing):
    """
    Executes a forward pass through the whole model.

    :param encoder:
    :param decoder:
    :param x: input to the encoder, shape [batch, seq_in_len]
    :param t: target output predictions for decoder, shape [batch, seq_t_len]
    :param criterion: loss function
    :param max_t_len: maximum target length

    :return: output (after log-softmax), loss, accuracy (per-symbol)
    """
    # Run encoder and get last hidden state (and output)
    #print(x)
    #print(len(x))
    batch_size = len(x)
    enc_h = encoder.init_hidden(batch_size)
    enc_out, enc_h = encoder(x, enc_h)

    dec_h = enc_h  # Init hidden state of decoder as hidden state of encoder
    dec_input = t_in
    out = decoder(dec_input, dec_h, max_t_len, teacher_forcing)
    out = out.permute(0, 2, 1)
    # Shape: [batch_size x num_classes x out_sequence_len], with second dim containing log probabilities

    loss = criterion(out, t)
    pred = get_pred(log_probs=out)
    accuracy = (pred == t).type(torch.FloatTensor).mean()
    return out, loss, accuracy


In [0]:
def train(encoder, decoder, inputs, targets, targets_in, criterion, enc_optimizer, dec_optimizer, epoch, max_t_len):
    encoder.train()
    decoder.train()
    for batch_idx, (x, t, t_in) in enumerate(zip(inputs, targets, targets_in)):
 
        x = torch.LongTensor(x).to(device)
        t = torch.LongTensor(t).to(device)
        t_in = torch.LongTensor(t_in).to(device)

        if len(x.shape) != 2:
          print(x.shape)
          print(x,t,t_in)
          raise
       # print(batch_idx)
#         inputs = inputs.to(device)
#         targets = targets.long()
#         targets_in = targets_in.long()
        out, loss, accuracy = forward_pass(encoder, decoder, x, t, t_in, criterion, max_t_len,
                                           teacher_forcing=TEACHER_FORCING)
        enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()
        loss.backward()
        enc_optimizer.step()
        dec_optimizer.step()
        if batch_idx % 200 == 0:
            print('Epoch {} [{}/{} ({:.0f}%)]\tTraining loss: {:.4f} \tTraining accuracy: {:.1f}%'.format(
                epoch, batch_idx * len(x), TRAINING_SIZE,
                100. * batch_idx * len(x) / TRAINING_SIZE, loss.item(),
                100. * accuracy.item()))


In [0]:
def test(encoder, decoder, inputs, targets, targets_in, criterion, max_t_len):
    encoder.eval()
    decoder.eval()
    with torch.no_grad():
#         inputs = inputs.to(device)
#         print(targets)
#         targets = targets.long()
#         targets_in = targets_in.long()
        inputs = inputs.view(inputs.shape[1],inputs.shape[2])
        targets = targets.view(targets.shape[1],targets.shape[2])
        targets_in = targets_in.view(targets_in.shape[1],targets_in.shape[2])


        out, loss, accuracy = forward_pass(encoder, decoder, inputs, targets, targets_in, criterion, max_t_len,
                                           teacher_forcing=TEACHER_FORCING)
    return out, loss, accuracy

In [0]:
def numbers_to_text(seq):
    return "".join([str(to_np(i)) if to_np(i) != 10 else '#' for i in seq])

def to_np(x):
    return x.cpu().numpy()

def get_pred(log_probs):
    """
    Get class prediction (digit prediction) from the net's output (the log_probs)
    :param log_probs: Tensor of shape [batch_size x n_classes x sequence_len]
    :return:
    """
    return torch.argmax(log_probs, dim=1)

In [153]:
encoder = EncoderRNN(NUM_INPUTS, NUM_UNITS_ENC,N_LAYERS,DROPOUT).to(device)
decoder = DecoderRNN(NUM_UNITS_DEC, NUM_OUTPUTS,N_LAYERS,DROPOUT).to(device)
enc_optimizer = optim.RMSprop(encoder.parameters(), lr=LEARNING_RATE)
dec_optimizer = optim.RMSprop(decoder.parameters(), lr=LEARNING_RATE)
criterion = nn.NLLLoss()

# Get training set
inputs, _, targets_in, targets, targets_seqlen, _, text, _, text_targ = generate(NUM_OF_BATCHES,TEST_SIZE, min_len=MIN_SEQ_LEN, max_len=MAX_SEQ_LEN)
max_target_len = max(targets_seqlen)
#inputs = torch.tensor(inputs)
# inputs = torch.LongTensor(inputs)
# targets = torch.LongTensor(targets)
# targets_in = torch.LongTensor(targets_in)
unique_text_targets = set([i for x in text_targ for i in x])

# Get validation set
val_inputs, _, val_targets_in, val_targets, val_targets_seqlen, _, val_text_in, _, val_text_targ = \
    generate(1,TEST_SIZE, min_len=MIN_SEQ_LEN, max_len=MAX_SEQ_LEN, invalid_set=unique_text_targets)
print(val_text_in, val_text_targ )
print( len(val_text_in), len(val_text_targ) )
print([len(x) for x in val_text_in])
#val_inputs = torch.tensor(val_inputs)
val_inputs = torch.LongTensor(val_inputs).to(device)
val_targets = torch.LongTensor(val_targets).to(device)
val_targets_in = torch.LongTensor(val_targets_in).to(device)
max_val_target_len = max(val_targets_seqlen)


# Quick and dirty - just loop over training set without reshuffling
for epoch in range(1, EPOCHS + 1):
    #print(inputs.shape)
    #print(len(inputs[0][0]))
    train(encoder, decoder, inputs, targets, targets_in, criterion, enc_optimizer, dec_optimizer, epoch, max_target_len)
    print('trained that shit')
    _, loss, accuracy = test(encoder, decoder, val_inputs, val_targets, val_targets_in, criterion, max_val_target_len)
    print('\nTest set: Average loss: {:.4f} \tAccuracy: {:.3f}%\n'.format(loss, accuracy.item()*100.))

    # Show examples
    print("Examples: prediction | input")
    out, _, _ = test(encoder, decoder, val_inputs[:10], val_targets[:10], val_targets_in[:10], criterion, max_val_target_len)
    pred = get_pred(out)
    pred_text = [numbers_to_text(sample) for sample in pred]
    print(len(pred_text),len(val_text_in),out.shape)
    for i in range(10):
        print(pred_text[i], "\t", val_text_in[0][i])
    print()

batch  0  starting
batch  0  ended with  100
batch  1  starting
batch  1  ended with  100
batch  2  starting
batch  2  ended with  100
batch  3  starting
batch  3  ended with  100
batch  4  starting
batch  4  ended with  100
batch  5  starting
batch  5  ended with  100
batch  6  starting
batch  6  ended with  100
batch  7  starting
batch  7  ended with  100
Generated batch length 8 from 100 iterations
batch  0  starting
batch  0  ended with  100
Generated batch length 1 from 100 iterations
[['seven seven nine six nine zero', 'nine four nine seven four five three', 'three seven four five six', 'three zero eight eight three four', 'six nine zero three five six zero three', 'three four five two seven', 'eight nine three zero eight nine seven five', 'eight seven five one seven three', 'seven nine eight nine seven three', 'zero six two zero three', 'nine six seven three one five', 'one zero five two seven six four one', 'nine seven three six two', 'five eight two six three seven', 'zero fou

# Exercise:

1. Implement missing code for the network in the *train* function. 
2. These networks implement the GRU-gates. Implement an alternative control utilising a memory mechanism (Hint: LSTM). What do you experience? 
3. (optional) There are some parameters in the model that may be optimized further, what could they be? Achieve >90% validation accuracy.